In [1]:
import pandas as pd
import yfinance as yf
from datetime import date, timedelta
from pandas.tseries.offsets import BMonthEnd, BMonthBegin
from sklearn.linear_model import LinearRegression
import numpy as np

In [2]:
def wrangle(stocks):
    #last day from previous month
    lday_pvm = pd.to_datetime(date.today() + BMonthEnd(-1))
    #first day from 4th previous months
    fday_4pvm = pd.to_datetime(date.today() + BMonthBegin(-4))
    df = yf.download(stocks, end=date.today())
    data = df[fday_4pvm:lday_pvm]["Adj Close"]
    return data

In [3]:
def fit_reg(stock_data):
    X = np.asarray(range(0, len(stock_data.index))).reshape(-1, 1)
    y = np.log1p(stock_data)
    reg = LinearRegression()
    reg.fit(X, y)
    return reg.coef_[0], reg.score(X, y)

In [4]:
import requests
import json

In [5]:
str_date = pd.to_datetime(date.today() + BMonthEnd(-1)).strftime('%Y%m%d')

In [6]:
url = f"https://www.blackrock.com/br/products/251816/ishares-ibovespa-fundo-de-ndice-fund/1506433276998.ajax?tab=all&fileType=json&asOfDate={str_date}&_=1660695636657"

In [7]:
page = requests.get(url)
decoded_data=page.text.encode().decode('utf-8-sig') 
data = json.loads(decoded_data)

In [8]:
stocks = []
for d in data["aaData"]:
    #include only equity assets (excluding futures, cash, cash funds etc)
    if d[10] == "XBSP":
        stocks.append(str(d[0]) + ".SA")

In [9]:
df = wrangle(stocks)

[*********************100%***********************]  90 of 90 completed


In [10]:
list_coef = []
list_score = []
for d in df:
    coef, score = fit_reg(df[d])
    list_coef.append(coef)
    list_score.append(score)
momentum = [x*y for x, y in zip(list_coef, list_score)]

In [11]:
momentum_stocks = pd.DataFrame(
    {
    "lr_coef": list_coef,
    "lr_r2": list_score,
    "momentum": momentum
    },
    index=df.columns
).sort_values("momentum", ascending=False).head(5).index.to_list()

In [15]:
first = yf.download(momentum_stocks, start=pd.to_datetime(date.today() + BMonthBegin(-1)), end=date.today())["Open"].iloc[0]

[*********************100%***********************]  5 of 5 completed


In [16]:
last = yf.download(momentum_stocks, start=pd.to_datetime(date.today() + BMonthBegin(-1)), end=date.today())["Adj Close"].iloc[-1]

[*********************100%***********************]  5 of 5 completed


In [33]:
round(((last - first) / first).mean()*100, 2)

7.69

In [35]:
def send_bot_message(text):
    base_url = f"https://api.telegram.org/bot5311633103:AAGa9I1UvslJjRNESegHSU6-xlPtoe7hKPE/sendMessage?chat_id=-1001153572177&text={text}"
    return requests.get(base_url).text

In [38]:
send_bot_message(momentum_stocks)

'{"ok":true,"result":{"message_id":5,"from":{"id":5311633103,"is_bot":true,"first_name":"MomentumBot","username":"momentumbr_bot"},"chat":{"id":-1001153572177,"title":"Momentum_BR","username":"momentum_br","type":"supergroup"},"date":1660766558,"text":"[\'CIEL3.SA\', \'ELET6.SA\', \'BRFS3.SA\', \'ELET3.SA\', \'HYPE3.SA\']","entities":[{"offset":2,"length":8,"type":"url"},{"offset":14,"length":8,"type":"url"},{"offset":26,"length":8,"type":"url"},{"offset":38,"length":8,"type":"url"},{"offset":50,"length":8,"type":"url"}]}}'

In [39]:
send_bot_message(str(round(((last - first) / first).mean()*100, 2)) + "%")

'{"ok":true,"result":{"message_id":6,"from":{"id":5311633103,"is_bot":true,"first_name":"MomentumBot","username":"momentumbr_bot"},"chat":{"id":-1001153572177,"title":"Momentum_BR","username":"momentum_br","type":"supergroup"},"date":1660766560,"text":"7.69%"}}'